# 使用 Meta 家族模型建構項目

## 簡介

本課程將會講解：

- 探索 Meta 家族兩款主要模型 —— Llama 3.1 及 Llama 3.2
- 了解每個模型的使用情境及適用場景
- 透過程式碼範例展示各模型的獨特功能

## Meta 家族模型

在本課程中，我們會探討 Meta 家族（又稱 "Llama Herd"）的兩款模型 —— Llama 3.1 及 Llama 3.2

這些模型有不同的版本，並可於 Github Model marketplace 取得。你可以參考以下連結，了解如何利用 Github Models [原型設計 AI 模型](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)。

模型版本：
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*注意：Llama 3 亦可於 Github Models 取得，但本課程不會涵蓋該模型*


## Llama 3.1

擁有4050億參數的Llama 3.1，屬於開源大型語言模型（LLM）類別。

這個版本是對早前Llama 3的升級，帶來以下改進：

- 更大的上下文視窗 —— 128k tokens 對比 8k tokens
- 更高的最大輸出tokens —— 4096 對比 2048
- 更佳的多語言支援 —— 因為訓練tokens數量增加

這些改進令Llama 3.1在開發生成式AI應用時，可以應付更複雜的情境，包括：
- 原生函數調用 —— 可以在LLM流程以外調用外部工具和函數
- 更佳的RAG表現 —— 因為上下文視窗更大
- 合成數據生成 —— 能夠為微調等任務創建有效數據


### 原生函數調用

Llama 3.1 已經經過微調，令佢喺調用函數或者工具時更加有效。佢內置咗兩個工具，模型可以根據用戶嘅提示自動識別需要用邊個工具。呢兩個工具分別係：

- **Brave Search** - 可以用嚟進行網絡搜尋，獲取最新資訊，例如天氣
- **Wolfram Alpha** - 可以用嚟做更複雜嘅數學計算，咁你就唔使自己寫函數

你都可以自訂屬於自己嘅工具，等 LLM 可以調用。

喺下面嘅程式碼例子：

- 我哋喺 system prompt 裡面定義咗可用嘅工具（brave_search, wolfram_alpha）。
- 發送一個用戶提示，查問某個城市嘅天氣。
- LLM 會回應一個工具調用，使用 Brave Search 工具，格式會係咁 `<|python_tag|>brave_search.call(query="Stockholm weather")`

*注意：呢個例子只係做咗工具調用，如果你想攞到結果，你需要去 Brave API 頁面註冊一個免費帳戶，並且自己定義個 function*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

雖然 Llama 3.1 已經係一個大型語言模型，但佢有個限制就係唔支援多模態。即係話，佢唔可以用唔同類型嘅輸入（例如圖片）嚟做提示，然後再作出回應。Llama 3.2 就加入咗呢個功能，成為佢其中一個主要賣點。其他新功能仲包括：

- 多模態支援 — 可以同時處理文字同圖片提示
- 細至中型模型選擇（11B 同 90B）— 提供更靈活嘅部署選項
- 只支援文字嘅版本（1B 同 3B）— 可以用喺邊緣／流動裝置，延遲更低

多模態支援對開源模型嚟講係一大突破。下面嘅程式碼例子會同時用圖片同文字提示，等 Llama 3.2 90B 幫你分析張圖。

### Llama 3.2 多模態支援


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## 學習唔止於此，繼續你嘅旅程

完成咗呢堂之後，可以去睇下我哋嘅 [生成式 AI 學習合集](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst)，繼續提升你對生成式 AI 嘅認識！



---

**免責聲明**：  
本文件經由 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能會出現錯誤或不準確之處。原始語言的文件應被視為具權威性的來源。如涉及重要資訊，建議尋求專業人手翻譯。本翻譯所引致的任何誤解或曲解，我們概不負責。
